# MSA 2023 Phase 2 - Part 3

## Deploy Model Endpoint

I deployed my hyperparameter tuned random forest model as an endpoint.

In [1]:
import urllib.request
import json
import os
import ssl

def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

allowSelfSignedHttps(True) # this line is needed if you use self-signed certificate in your scoring service.

# Request data goes here
# The example below assumes JSON formatting which may be updated
# depending on the format your endpoint expects.
# More information can be found here:
# https://docs.microsoft.com/azure/machine-learning/how-to-deploy-advanced-entry-script

# 16 input features
data = {
    "data": [
        [1]*16,
        [0]*16
    ]
}

body = str.encode(json.dumps(data))

url = 'https://msa-phase2-azure-wkmks.australiaeast.inference.ml.azure.com/score'
# Replace this with the primary/secondary key or AMLToken for the endpoint
api_key = 'nthTxP41eyqSgslpIUMqVo6qaIKqbkIx'
if not api_key:
    raise Exception("A key should be provided to invoke the endpoint")

# The azureml-model-deployment header will force the request to go to a specific deployment.
# Remove this header to have the request observe the endpoint traffic rules
headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key), 'azureml-model-deployment': 'segmentation-rf-4' }

req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)

    result = response.read()
    print(result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))

    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(error.read().decode("utf8", 'ignore'))

b'["A", "D"]'


## Run Training Experiments

In [Run Experiment](run-experiment.ipynb), I created parametised training scripts for the logistic regression model and the random forest model for market segmentation.

The logistic regression training script accepted the following parameters:
- Regularisation rate
- Solver

The random forest training script accepeted the following parameters:
- Maximum depth of the tree
- Maximum number of features considered at each split
- Minimum number of samples required for a split
- Minimum number of samples per leaf node

I then ran the experiments as jobs on Azure using a compute, passing in my own parameters.

### Logistic Regression

![](images/lg-experiment.png)

### Random Forest

![](images/rf-experiment.png)

## Tune Hyperparameters

In [Tune Hyperparameters](tune-hyperparameters.ipynb), I created hyperparameter training scripts for the logistic regression model and the random forest model for market segmentation, with parameters as described in the above section.

I then ran hyperparameter tuning experiments with grid sampled hyperparameters, then determined which parameters lead to the best run. I continued refining the hyperparameters until I couldn't obtain a significant improvement in the accuracy.

I found that although the logistic regression model had a higher accuracy in Part 2 than the random forest model with the default settings, the random forest model was able to achieve much higher accuracy than the logistic regression model with hyperparameter tuning. I found that hyperparameter tuning made no significant improvement to the logistic regression model, whereas it made significant improvement to the random forest model.

### Logistic Regression

#### Final Run

![](images/parallel-coordinates-lg-1.png)
![](images/accuracy-table-lg-1.png)

There is very little effect on accuracy across the hyperparameter space, therefore we stop hyperparameter tuning.

#### Best Metrics

![](images/best-metrics-lg-1.png)

### Random Forest

#### Run 1

![](images/parallel-coordinates-rf-1.png)
![](images/accuracy-table-rf-1.png)

#### Run 2

![](images/parallel-coordinates-rf-2.png)
![](images/accuracy-table-rf-2.png)

#### Final Run

![](images/accuracy-table-rf-3.png)
![](images/parallel-coordinates-rf-3.png)

There appears to be no improvement in the accuracy in further iterations of hyperparameter tuning, therefore we stop hyperparameter tuning.

#### Best Metrics

![](images/best-metrics-rf-1.png)